In [ ]:
import pandas as pd

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://sec

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.ml.feature import StringIndexer,VectorAssembler

spark = SparkSession.builder\
            .appName("Spark Dataframe Intro")\
            .getOrCreate()

In [ ]:
leaf = spark.read.option("header","false").option("inferSchema","true").csv('/content/leaf.csv')

In [ ]:
leaf.show()

+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+--------+---------+---------+---------+-------+
|_c0|_c1|    _c2|   _c3|    _c4|    _c5|    _c6|    _c7|      _c8|      _c9|     _c10|    _c11|     _c12|     _c13|     _c14|   _c15|
+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+--------+---------+---------+---------+-------+
|  1|  1|0.72694|1.4742|0.32396|0.98535|    1.0|0.83592|0.0046566|0.0039465|  0.04779| 0.12795| 0.016108|0.0052323|2.7477E-4| 1.1756|
|  1|  2|0.74173|1.5257|0.36116|0.98152|0.99825|0.79867|0.0052423|0.0050016|  0.02416|0.090476|0.0081195| 0.002708|7.4846E-5|0.69659|
|  1|  3|0.76722|1.5725|0.38998|0.97755|    1.0|0.80812|0.0074573| 0.010121| 0.011897|0.057445|0.0032891|9.2068E-4|3.7886E-5|0.44348|
|  1|  4|0.73797|1.4597|0.35376|0.97566|    1.0|0.81697|0.0068768|0.0086068|  0.01595|0.065491|0.0042707|0.0011544|6.6272E-5|0.58785|
|  1|  5|0.82301|1.7707|0.44462|0.97698|    1.0|0.75493| 0.007

In [ ]:
leaf.select("_c0").distinct().show(10000)

# Too many target exists.. Precision might not be high

+---+
|_c0|
+---+
| 31|
| 34|
| 28|
| 26|
| 27|
| 12|
| 22|
|  1|
| 13|
|  6|
|  3|
|  5|
| 15|
|  9|
| 35|
|  4|
|  8|
| 23|
|  7|
| 10|
| 25|
| 24|
| 29|
| 32|
| 11|
| 33|
| 14|
|  2|
| 30|
| 36|
+---+



In [ ]:
# As I understand from the datasets ReadMe paper, columns 11 to 16 are statistical informations which are calculated from other columns of datasets.
# So if these are calculated with other columns value, we dont need to keep these values in datasets. That might increase our model complexiticy redundantly.
  

In [ ]:
leaf_new = leaf.drop('_c11','_c12','_c13','_c14','_c15')

In [ ]:
leaf_new.show()

+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+
|_c0|_c1|    _c2|   _c3|    _c4|    _c5|    _c6|    _c7|      _c8|      _c9|     _c10|
+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+
|  1|  1|0.72694|1.4742|0.32396|0.98535|    1.0|0.83592|0.0046566|0.0039465|  0.04779|
|  1|  2|0.74173|1.5257|0.36116|0.98152|0.99825|0.79867|0.0052423|0.0050016|  0.02416|
|  1|  3|0.76722|1.5725|0.38998|0.97755|    1.0|0.80812|0.0074573| 0.010121| 0.011897|
|  1|  4|0.73797|1.4597|0.35376|0.97566|    1.0|0.81697|0.0068768|0.0086068|  0.01595|
|  1|  5|0.82301|1.7707|0.44462|0.97698|    1.0|0.75493| 0.007428| 0.010042|0.0079379|
|  1|  6|0.72997|1.4892|0.34284|0.98755|    1.0|0.84482|0.0049451|0.0044506| 0.010487|
|  1|  7|0.82063|1.7529|0.44458|0.97964|0.99649| 0.7677|0.0059279|0.0063954| 0.018375|
|  1|  8|0.77982|1.6215|0.39222|0.98512|0.99825|0.80816|0.0050987|0.0047314| 0.024875|
|  1|  9|0.83089|1.8199|0.45693| 0.9824|   

In [ ]:
leaf_new.summary().show()  # I dont see any null values in data set. All columns count is 340..

+-------+------------------+------------------+-------------------+-----------------+------------------+-------------------+-------------------+-------------------+--------------------+------------------+--------------------+
|summary|               _c0|               _c1|                _c2|              _c3|               _c4|                _c5|                _c6|                _c7|                 _c8|               _c9|                _c10|
+-------+------------------+------------------+-------------------+-----------------+------------------+-------------------+-------------------+-------------------+--------------------+------------------+--------------------+
|  count|               340|               340|                340|              340|               340|                340|                340|                340|                 340|               340|                 340|
|   mean|18.544117647058822|6.2823529411764705| 0.7198541764705882|          2.44021|0.513760411

In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
#first i will try RandomForest:

In [ ]:
vec = VectorAssembler(inputCols=leaf_new.columns[2:],outputCol='features')
leaf_new = vec.transform(leaf_new)


In [ ]:
leaf_new.show()

+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+--------------------+
|_c0|_c1|    _c2|   _c3|    _c4|    _c5|    _c6|    _c7|      _c8|      _c9|     _c10|            features|
+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+--------------------+
|  1|  1|0.72694|1.4742|0.32396|0.98535|    1.0|0.83592|0.0046566|0.0039465|  0.04779|[0.72694,1.4742,0...|
|  1|  2|0.74173|1.5257|0.36116|0.98152|0.99825|0.79867|0.0052423|0.0050016|  0.02416|[0.74173,1.5257,0...|
|  1|  3|0.76722|1.5725|0.38998|0.97755|    1.0|0.80812|0.0074573| 0.010121| 0.011897|[0.76722,1.5725,0...|
|  1|  4|0.73797|1.4597|0.35376|0.97566|    1.0|0.81697|0.0068768|0.0086068|  0.01595|[0.73797,1.4597,0...|
|  1|  5|0.82301|1.7707|0.44462|0.97698|    1.0|0.75493| 0.007428| 0.010042|0.0079379|[0.82301,1.7707,0...|
|  1|  6|0.72997|1.4892|0.34284|0.98755|    1.0|0.84482|0.0049451|0.0044506| 0.010487|[0.72997,1.4892,0...|
|  1|  7|0.82063|1.7529|0.44

In [ ]:
leaf_vectorized = leaf_new.select('features','_c0')
leaf_vectorized = leaf_vectorized.withColumnRenamed('_c0','label')
leaf_vectorized.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.72694,1.4742,0...|    1|
|[0.74173,1.5257,0...|    1|
|[0.76722,1.5725,0...|    1|
|[0.73797,1.4597,0...|    1|
|[0.82301,1.7707,0...|    1|
|[0.72997,1.4892,0...|    1|
|[0.82063,1.7529,0...|    1|
|[0.77982,1.6215,0...|    1|
|[0.83089,1.8199,0...|    1|
|[0.90631,2.3906,0...|    1|
|[0.7459,1.4927,0....|    1|
|[0.79606,1.6934,0...|    1|
|[0.93361,2.7582,0...|    2|
|[0.91186,2.4994,0...|    2|
|[0.89063,2.2927,0...|    2|
|[0.86755,2.009,0....|    2|
|[0.91852,2.5247,0...|    2|
|[0.88795,2.2038,0...|    2|
|[0.85121,1.9548,0...|    2|
|[0.89084,2.2979,0...|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
trainDF , testDF = leaf_vectorized.randomSplit([0.7,0.3])

In [ ]:
rfclassifier = RandomForestClassifier()
params = ParamGridBuilder()\
      .addGrid(rfclassifier.maxDepth,[2,3,4,5,7,9,11])\
      .addGrid(rfclassifier.numTrees,[1,2,3,4,10,20,25,30,35])\
      .addGrid(rfclassifier.impurity,['gini','entropy'])\
      .build()
eva = MulticlassClassificationEvaluator(metricName='accuracy')
validator = CrossValidator(estimatorParamMaps=params,
                                 estimator=rfclassifier,
                                 evaluator=eva,                                 
                                 parallelism=32         )

model = validator.fit(trainDF)
print("Num Trees : ",model.bestModel.getNumTrees)
print("Max Depth : ",model.bestModel._java_obj.getMaxDepth())
print("Impurtiy : ",model.bestModel._java_obj.getImpurity())

sonucDF = model.transform(testDF)
basari = eva.evaluate(sonucDF)
print("Başarı : ",basari)

Num Trees :  30
Max Depth :  11
Impurtiy :  gini
Başarı :  0.6237623762376238


In [ ]:
# Hmm, 62 percent not that bad for many classes..

#I want to try a parametric model, 
#Logistic Regggg:

In [ ]:
# Let us scale the columns first:

In [ ]:
trainDF , testDF = leaf_vectorized.randomSplit([0.7,0.3])
#First split, then scale..

In [ ]:
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
scaled_vec_train=standardscaler.fit(trainDF).transform(trainDF)
scaled_vec_test =standardscaler.fit(trainDF).transform(testDF)
scaled_vec_train=scaled_vec_train.select("Scaled_features","label")
scaled_vec_test=scaled_vec_test.select("Scaled_features","label")

In [ ]:
scaled_vec_test.show()

+--------------------+-----+
|     Scaled_features|label|
+--------------------+-----+
|[0.70849172706039...|   36|
|[1.15662952772804...|   11|
|[1.32677911572286...|   36|
|[1.52080354171084...|   10|
|[1.57214626262286...|   30|
|[1.84814884520518...|   11|
|[1.84819612211209...|   36|
|[1.87244917536059...|   30|
|[2.03716191905994...|    9|
|[2.05299968287718...|   11|
|[2.09299594612909...|   15|
|[2.12188213625547...|   11|
|[2.17918174743905...|   10|
|[2.17937085506672...|    9|
|[2.20910802951761...|   10|
|[2.22787696156372...|   36|
|[2.25643221334167...|    6|
|[2.26054530424347...|    6|
|[2.27983428226566...|   26|
|[2.36923491324599...|   10|
+--------------------+-----+
only showing top 20 rows



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[lr])
params = ParamGridBuilder()\
    .addGrid(lr.regParam, [0, 0.01, 0.05, 0.1,0.2,0.35, 0.5, 1,2,3,5,7,10])\
    .addGrid(lr.elasticNetParam, [0.0, 0.1,0.2,0.3, 0.5, 0.8, 1])\
    .build()



evaluator = BinaryClassificationEvaluator(metricName = 'accuracy')
validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=lr,
                                evaluator=eva   ,
                                numFolds=5        )
                          
model = validator.fit(trainDF)
sonucDF = model.transform(testDF)
basari = eva.evaluate(sonucDF)

sonucDF = model.transform(testDF)
basari = eva.evaluate(sonucDF)

print("Başarı : ",basari)

Başarı :  0.5925925925925926


In [ ]:
scaled_vec_train.show

In [ ]:
# These accuracy rates make me upset :((()))

#I want to try NaiveBayes model, our data set is very restricted. NB may help..

In [ ]:
trainDF , testDF = leaf_vectorized.randomSplit([0.7,0.3])
#First split, then scale..
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
scaled_vec_train=standardscaler.fit(trainDF).transform(trainDF)
scaled_vec_test =standardscaler.fit(trainDF).transform(testDF)
scaled_vec_train=scaled_vec_train.select("Scaled_features","label")
scaled_vec_test=scaled_vec_test.select("Scaled_features","label")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.feature import Tokenizer,Word2Vec,StopWordsRemover,NGram,CountVectorizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier,RandomForestClassifier,NaiveBayes
spark = SparkSession.builder.appName("Spark Dataframe Null Values").getOrCreate()



classifier = NaiveBayes() 
#trainDF,testDF = textDF.randomSplit([0.75,0.25])
model = classifier.fit(trainDF)
resultDF = model.transform(testDF)
eva = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = eva.evaluate(resultDF)
print('accuracy:',accuracy)

accuracy: 0.03409090909090909


Noppee, NB did not give hand to us.. 3 percent accuracy..

In [ ]:
f1_score = MulticlassClassificationEvaluator(metricName="f1")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier,NaiveBayes
decision_tree= DecisionTreeClassifier()

params = ParamGridBuilder()\
    .addGrid(decision_tree.maxDepth, [4,5,10,15,20]) \
    .addGrid(decision_tree.maxBins, [10,20,25,35])\
    .addGrid(decision_tree.impurity, ['gini',"entropy"])\
    .build()

dc_validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=decision_tree,
                                evaluator=f1_score,
                                numFolds=5,
                                parallelism=4
                          )

dc_model = dc_validator.fit(trainDF)

print("Impurtiy : ",dc_model.bestModel._java_obj.getImpurity())

result = dc_model.transform(testDF)

accuracy_result = MulticlassClassificationEvaluator(metricName="accuracy").evaluate(result)
#log_accuracy.append(('DecisionTreeClassifier',accuracy_result))
#print("Accuracy : ",accuracy_result)
print('accuracy_result:',accuracy_result)

Impurtiy :  gini
accuracy_result: 0.4431818181818182
